#### Final Mass Budget 

In [1]:
#importing the libraries required for the evaluation
import numpy as np
import rasterio
from pygeotools.lib import malib, warplib, iolib,geolib, timelib
import numpy.ma as ma
%matplotlib inline
import matplotlib.pyplot as plt
import os
import pandas as pd
from imview.lib import pltlib
from osgeo import gdal, ogr

### Calculate mass budget for glaciers having more than 80% coverage and area greater than 0.1 sqkm.

In [2]:
glac_shp_fn='mummy.shp'
#ogr2ogr -f "ESRI Shapefile" 15_rgi50_SouthAsiaEast.shp  -t_srs EPSG:32645 in Bash Commandline to change projection of vectors
glac_shp_ds = ogr.Open(glac_shp_fn, 0) #shapefile_dataset
glac_shp_lyr = glac_shp_ds.GetLayer() #layer of shapefile, all the operations are performed on it....
glac_shp_srs = glac_shp_lyr.GetSpatialRef() #Spatial_reference_of_shapefile
feat_count = glac_shp_lyr.GetFeatureCount() #Feature_Count
print("Input glacier polygon count: %i" % feat_count)
print str(glac_shp_srs)

Input glacier polygon count: 13119
PROJCS["WGS_1984_UTM_Zone_45N",
    GEOGCS["GCS_WGS_1984",
        DATUM["WGS_1984",
            SPHEROID["WGS_84",6378137,298.257223563]],
        PRIMEM["Greenwich",0],
        UNIT["Degree",0.017453292519943295],
        AUTHORITY["EPSG","4326"]],
    PROJECTION["Transverse_Mercator"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",87],
    PARAMETER["scale_factor",0.9996],
    PARAMETER["false_easting",500000],
    PARAMETER["false_northing",0],
    UNIT["Meter",1],
    AUTHORITY["EPSG","32645"]]


In [3]:
#dh_dt='WV_Carto_RateMaps-tile-0-last.tif' #input elevation_difference_map
dh_dt='Filtered_dh_1_5_IQR.tif' #input elevation_difference_map
z='B-tile-0.tif' #input DEM
dh_dt_ds=gdal.Open(dh_dt) 
z_ds=gdal.Open(z)
dz_int_geom = geolib.ds_geom_intersection([dh_dt_ds, z_ds], t_srs=glac_shp_srs) #intersecting_extent
glac_shp_lyr.SetSpatialFilter(dz_int_geom) #spatial filtering of shapefiles within extent
feat_count = glac_shp_lyr.GetFeatureCount() #now only the polygons within the extents are considered
print("Filtered glacier polygon count: %i" % feat_count)
glac_shp_lyr.ResetReading()

Filtered glacier polygon count: 1072


In [4]:
glacname_fieldname = "Name"
glacnum_fieldname = "RGIId"
#Approach inspired from David's mb.py script
#Create polygon for valid pixels in DEM mosaics
#Intersect with each glacier polygon, only preserve those with 80% overlap
#Go through shp, extract feat name/number and geom
min_glac_area = 0.1
i=0
for n, feat in enumerate(glac_shp_lyr):
    print'Hi' #Check to see program flow...
    glacname = feat.GetField(glacname_fieldname)
    #Extracting glacier id/name from input shapefiles
    if glacname is None:
        glacname = ""
    glacnum = feat.GetField(glacnum_fieldname)
    #RGIId (String) = RGI50-01.00004
    #glacnum = float(glacnum.split('-')[-1])*10000000
    glacnum = float(glacnum.split('-')[-1])
    glacnum_fmt = '%0.5f'
    if glacname:
        feat_fn = "%s_%s" % (glacnum, glacname)
    else:
        feat_fn = str(glacnum)
    print("\n%i of %i: %s\n" % (n+1, feat_count, feat_fn))
    #extracting feature geometry, area, if less than 0.1sq.km., ignoring it
    glac_geom = feat.GetGeometryRef()
    glac_geom.AssignSpatialReference(glac_shp_srs)
    glac_geom_extent = geolib.geom_extent(glac_geom)
    glac_area = glac_geom.GetArea()
    print str(glac_area)
    if (glac_area/1E6) < min_glac_area:
        print("Glacier area below %0.1f km2 threshold" % min_glac_area)
        continue
    #warping the dh_dt and elevation map into common extents
    ds_list = warplib.memwarp_multi_fn([dh_dt, z], res='max',r='average',
            extent=glac_geom_extent, t_srs='EPSG:32645', verbose=False)
    #Defining mask based on geometry of current feature in loop
    glac_geom_mask = geolib.geom2mask(glac_geom, ds_list[0])
    dh_dt_a = ma.array(iolib.ds_getma(ds_list[0]), mask=glac_geom_mask)
    z_a = ma.array(iolib.ds_getma(ds_list[1]), mask=glac_geom_mask) 
    #80 percent_area_quantification
    ds_res = geolib.get_res(ds_list[0])
    valid_area = dh_dt_a.count()*ds_res[0]*ds_res[1]
    valid_area_perc = valid_area/glac_area
    min_valid_area_perc = 0.80
    print'Ho' #Check to see program flow...
    if valid_area_perc < min_valid_area_perc:
        print("Not enough valid pixels. %0.1f%% percent of glacier polygon area" % (100*valid_area_perc))
        continue
    #preparing dataframes for qualifying glaciers
    common_mask=malib.common_mask([dh_dt_a,z_a]) #common mask based on DEM and dh_dt as there might be some DEM pixels for no dh/dt pixel
    dh_dt_f=ma.compressed(ma.array(dh_dt_a,mask=common_mask))
    z_f=ma.compressed(ma.array(z_a,mask=common_mask))
    raw={'Elevation':z_f,
    feat_fn:dh_dt_f} #feat_fn saves the rgi id or name of the glacier
    print'Hola' #Check to see program flow...
    df1=pd.DataFrame(raw, columns=['Elevation',feat_fn])
    #Sorting the obtained dataframes based on asscending elevation
    df1=df1.sort_values(by='Elevation',ascending=True)
    #Rounding Glacier Elevation to next nearest multiple of 50
    df1['Elevation_rounded']=(df1['Elevation']+49)//50*50
    #Calulating_stats_for_Glacier_Pixels
    grouped=df1.groupby('Elevation_rounded')
    df2=grouped[feat_fn].agg([np.mean,np.count_nonzero,np.median])
    #Creating a coulumn identical to the indexed "Elevation rouded" so that it can be used for manipulation
    df2['E1'] = df2.index
    #znorm= (z - zmin) /(zmax-zmin) #Larsen et al. 2015, GRL
    #DataFrame.max(axis=None, skipna=None, level=None, numeric_only=None, **kwargs) #Syntax
    Max_limit=df2['E1'].max()
    print Max_limit
    Min_limit=df2['E1'].min()
    print Min_limit
    deno=Max_limit-Min_limit
    df2['Elevation_normalised']=(df2['E1']-Min_limit)/deno
    df2['Area']=df2['count_nonzero']*ds_res[0]*ds_res[1]
    df3=df2[['Area', 'mean', 'Elevation_normalised','median']].copy()
    df3['RGI_Id']=feat_fn
    if i==0:
        df = df3
    else:
        df = df.append(df3)
    i=i+1

Hi

1 of 1072: 15.02697

526041.931846

Warping all inputs to the following:
Resolution: 10.0
Extent: [615955.0087304136, 3085337.401336626, 617211.4534322533, 3085943.840496815]
Projection: '+proj=utm +zone=45 +datum=WGS84 +units=m +no_defs '
Resampling alg: average

1 of 2: Filtered_dh_1_5_IQR.tif
nl: 61 ns: 126 res: 10.000
2 of 2: B-tile-0.tif
nl: 61 ns: 126 res: 10.000
Ho
Hola
5750.0
5550.0
Hi

2 of 1072: 15.02698

11704364.2729

Warping all inputs to the following:
Resolution: 10.0
Extent: [611049.993403404, 3084082.156530921, 617563.6785395433, 3088393.8870531027]
Projection: '+proj=utm +zone=45 +datum=WGS84 +units=m +no_defs '
Resampling alg: average

1 of 2: Filtered_dh_1_5_IQR.tif
nl: 431 ns: 651 res: 10.000
2 of 2: B-tile-0.tif
nl: 431 ns: 651 res: 10.000
Ho
Hola
7450.0
5200.0
Hi

3 of 1072: 15.02699

3009359.61128

Warping all inputs to the following:
Resolution: 10.0
Extent: [609987.3776952798, 3082838.6865264266, 612237.2224962701, 3084719.877870378]
Projection: '+proj=utm

2 of 2: B-tile-0.tif
nl: 181 ns: 158 res: 10.000
Ho
Not enough valid pixels. 0.0% percent of glacier polygon area
Hi

29 of 1072: 15.02725

114489.043732

Warping all inputs to the following:
Resolution: 10.0
Extent: [605387.6771417009, 3046816.5957449283, 605830.6931298929, 3047304.129839508]
Projection: '+proj=utm +zone=45 +datum=WGS84 +units=m +no_defs '
Resampling alg: average

1 of 2: Filtered_dh_1_5_IQR.tif
nl: 49 ns: 44 res: 10.000
2 of 2: B-tile-0.tif
nl: 49 ns: 44 res: 10.000
Ho
Not enough valid pixels. 0.0% percent of glacier polygon area
Hi

30 of 1072: 15.02726

119782.094543

Warping all inputs to the following:
Resolution: 10.0
Extent: [604045.6944284941, 3046171.4528893665, 604597.5162961292, 3046563.1008623894]
Projection: '+proj=utm +zone=45 +datum=WGS84 +units=m +no_defs '
Resampling alg: average

1 of 2: Filtered_dh_1_5_IQR.tif
nl: 39 ns: 55 res: 10.000
2 of 2: B-tile-0.tif
nl: 39 ns: 55 res: 10.000
Ho
Not enough valid pixels. 0.0% percent of glacier polygon area
Hi


Ho
Not enough valid pixels. 4.8% percent of glacier polygon area
Hi

56 of 1072: 15.02752

130971.092539

Warping all inputs to the following:
Resolution: 10.0
Extent: [620000.883736785, 3043631.5204792805, 620799.6060242348, 3043987.2267734795]
Projection: '+proj=utm +zone=45 +datum=WGS84 +units=m +no_defs '
Resampling alg: average

1 of 2: Filtered_dh_1_5_IQR.tif
nl: 36 ns: 80 res: 10.000
2 of 2: B-tile-0.tif
nl: 36 ns: 80 res: 10.000
Ho
Not enough valid pixels. 0.0% percent of glacier polygon area
Hi

57 of 1072: 15.02753

151732.280261

Warping all inputs to the following:
Resolution: 10.0
Extent: [620841.644012637, 3043401.9282897143, 621291.1274611555, 3043961.357201328]
Projection: '+proj=utm +zone=45 +datum=WGS84 +units=m +no_defs '
Resampling alg: average

1 of 2: Filtered_dh_1_5_IQR.tif
nl: 56 ns: 45 res: 10.000
2 of 2: B-tile-0.tif
nl: 56 ns: 45 res: 10.000
Ho
Not enough valid pixels. 1.4% percent of glacier polygon area
Hi

58 of 1072: 15.02754

402269.662646

Warping all i

Ho
Not enough valid pixels. 71.8% percent of glacier polygon area
Hi

76 of 1072: 15.02772

604596.12438

Warping all inputs to the following:
Resolution: 10.0
Extent: [595627.1369886842, 3059801.4429760254, 596471.1309436946, 3061086.5565208676]
Projection: '+proj=utm +zone=45 +datum=WGS84 +units=m +no_defs '
Resampling alg: average

1 of 2: Filtered_dh_1_5_IQR.tif
nl: 129 ns: 84 res: 10.000
2 of 2: B-tile-0.tif
nl: 129 ns: 84 res: 10.000
Ho
Hola
5500.0
5050.0
Hi

77 of 1072: 15.02773

110430.585231

Warping all inputs to the following:
Resolution: 10.0
Extent: [596632.0071377569, 3060318.8338335683, 596989.6894511122, 3061038.3306276384]
Projection: '+proj=utm +zone=45 +datum=WGS84 +units=m +no_defs '
Resampling alg: average

1 of 2: Filtered_dh_1_5_IQR.tif
nl: 72 ns: 36 res: 10.000
2 of 2: B-tile-0.tif
nl: 72 ns: 36 res: 10.000
Ho
Hola
5300.0
5000.0
Hi

78 of 1072: 15.02774

2243521.30217

Warping all inputs to the following:
Resolution: 10.0
Extent: [595505.3663213975, 3060499.9206


101 of 1072: 15.02797

52618.4833775
Glacier area below 0.1 km2 threshold
Hi

102 of 1072: 15.02798

286752.860222

Warping all inputs to the following:
Resolution: 10.0
Extent: [608480.9686945714, 3070216.848028187, 609760.7064836831, 3070731.2252871925]
Projection: '+proj=utm +zone=45 +datum=WGS84 +units=m +no_defs '
Resampling alg: average

1 of 2: Filtered_dh_1_5_IQR.tif
nl: 51 ns: 128 res: 10.000
2 of 2: B-tile-0.tif
nl: 51 ns: 128 res: 10.000
Ho
Not enough valid pixels. 20.8% percent of glacier polygon area
Hi

103 of 1072: 15.02799

831524.432947

Warping all inputs to the following:
Resolution: 10.0
Extent: [607487.2119170197, 3071146.5547509524, 609467.5182805737, 3071864.7113914913]
Projection: '+proj=utm +zone=45 +datum=WGS84 +units=m +no_defs '
Resampling alg: average

1 of 2: Filtered_dh_1_5_IQR.tif
nl: 72 ns: 198 res: 10.000
2 of 2: B-tile-0.tif
nl: 72 ns: 198 res: 10.000
Ho
Not enough valid pixels. 6.9% percent of glacier polygon area
Hi

104 of 1072: 15.028

833164.269

...

Ho
Not enough valid pixels. 0.0% percent of glacier polygon area
Hi

939 of 1072: 15.10574_CN5O198D0005

64452.2584454
Glacier area below 0.1 km2 threshold
Hi

940 of 1072: 15.10575_CN5O198D0004

139716.985788

Warping all inputs to the following:
Resolution: 10.0
Extent: [558515.970376562, 3099187.9485697607, 559067.3944671664, 3099518.0847044084]
Projection: '+proj=utm +zone=45 +datum=WGS84 +units=m +no_defs '
Resampling alg: average

1 of 2: Filtered_dh_1_5_IQR.tif
nl: 33 ns: 55 res: 10.000
2 of 2: B-tile-0.tif
nl: 33 ns: 55 res: 10.000
Ho
Not enough valid pixels. 0.0% percent of glacier polygon area
Hi

941 of 1072: 15.10576_CN5O198D0004

187863.066276

Warping all inputs to the following:
Resolution: 10.0
Extent: [558490.764453548, 3099690.942163248, 559244.1883520719, 3100487.1574185784]
Projection: '+proj=utm +zone=45 +datum=WGS84 +units=m +no_defs '
Resampling alg: average

1 of 2: Filtered_dh_1_5_IQR.tif
nl: 80 ns: 75 res: 10.000
2 of 2: B-tile-0.tif
nl: 80 ns: 75 res: 10.000


Ho
Not enough valid pixels. 0.0% percent of glacier polygon area
Hi

967 of 1072: 15.10602_CN5O198E0016

435917.758367

Warping all inputs to the following:
Resolution: 10.0
Extent: [559633.8670058906, 3100624.185703492, 560218.2302445981, 3101898.281902791]
Projection: '+proj=utm +zone=45 +datum=WGS84 +units=m +no_defs '
Resampling alg: average

1 of 2: Filtered_dh_1_5_IQR.tif
nl: 127 ns: 58 res: 10.000
2 of 2: B-tile-0.tif
nl: 127 ns: 58 res: 10.000
Ho
Not enough valid pixels. 0.0% percent of glacier polygon area
Hi

968 of 1072: 15.10603_CN5O198E0022 Amazhengmai Glacier

1587964.77888

Warping all inputs to the following:
Resolution: 10.0
Extent: [558655.2274967546, 3106907.265225286, 561741.2570306311, 3108164.357694149]
Projection: '+proj=utm +zone=45 +datum=WGS84 +units=m +no_defs '
Resampling alg: average

1 of 2: Filtered_dh_1_5_IQR.tif
nl: 126 ns: 309 res: 10.000
2 of 2: B-tile-0.tif
nl: 126 ns: 309 res: 10.000
Ho
Not enough valid pixels. 0.0% percent of glacier polygon area
H

Ho
Hola
6050.0
5600.0
Hi

993 of 1072: 15.10628_CN5O198E0093

118936.684682

Warping all inputs to the following:
Resolution: 10.0
Extent: [570867.7610251348, 3083113.069206015, 571449.5564410221, 3083603.923264088]
Projection: '+proj=utm +zone=45 +datum=WGS84 +units=m +no_defs '
Resampling alg: average

1 of 2: Filtered_dh_1_5_IQR.tif
nl: 49 ns: 58 res: 10.000
2 of 2: B-tile-0.tif
nl: 49 ns: 58 res: 10.000
Ho
Hola
6000.0
5500.0
Hi

994 of 1072: 15.10629_CN5O198E0106

346696.02069

Warping all inputs to the following:
Resolution: 10.0
Extent: [570856.9226568874, 3075825.196422441, 571554.2005730905, 3076618.6157322763]
Projection: '+proj=utm +zone=45 +datum=WGS84 +units=m +no_defs '
Resampling alg: average

1 of 2: Filtered_dh_1_5_IQR.tif
nl: 79 ns: 70 res: 10.000
2 of 2: B-tile-0.tif
nl: 79 ns: 70 res: 10.000
Ho
Hola
5400.0
5050.0
Hi

995 of 1072: 15.1063_CN5O198E0102

50417.316836
Glacier area below 0.1 km2 threshold
Hi

996 of 1072: 15.10631_CN5O198E0094

106115.343591

Warping all 

47196.3942422
Glacier area below 0.1 km2 threshold
Hi

1022 of 1072: 15.10657_CN5O198E0060

26310.0761808
Glacier area below 0.1 km2 threshold
Hi

1023 of 1072: 15.10658_CN5O198E0065

19062.3795064
Glacier area below 0.1 km2 threshold
Hi

1024 of 1072: 15.10659_CN5O198E0059

490151.585363

Warping all inputs to the following:
Resolution: 10.0
Extent: [577934.1376956869, 3090355.9177454426, 579069.0910901788, 3091135.75757115]
Projection: '+proj=utm +zone=45 +datum=WGS84 +units=m +no_defs '
Resampling alg: average

1 of 2: Filtered_dh_1_5_IQR.tif
nl: 78 ns: 113 res: 10.000
2 of 2: B-tile-0.tif
nl: 78 ns: 113 res: 10.000
Ho
Hola
5950.0
5450.0
Hi

1025 of 1072: 15.1066_CN5O198E0062

61446.5990007
Glacier area below 0.1 km2 threshold
Hi

1026 of 1072: 15.10661_CN5O198E0057

1498234.18345

Warping all inputs to the following:
Resolution: 10.0
Extent: [578443.7478423164, 3090825.6869382295, 580533.9938575509, 3092566.1823049965]
Projection: '+proj=utm +zone=45 +datum=WGS84 +units=m +no_defs 

Ho
Not enough valid pixels. 10.5% percent of glacier polygon area
Hi

1050 of 1072: 15.10685_CN5O201A0001

181025.067027

Warping all inputs to the following:
Resolution: 10.0
Extent: [683801.4723392916, 3070133.8423341117, 684353.8796222301, 3070725.717210378]
Projection: '+proj=utm +zone=45 +datum=WGS84 +units=m +no_defs '
Resampling alg: average

1 of 2: Filtered_dh_1_5_IQR.tif
nl: 59 ns: 55 res: 10.000
2 of 2: B-tile-0.tif
nl: 59 ns: 55 res: 10.000
Ho
Not enough valid pixels. 45.2% percent of glacier polygon area
Hi

1051 of 1072: 15.10686_CN5O201A0019

80082.8931554
Glacier area below 0.1 km2 threshold
Hi

1052 of 1072: 15.10687_CN5O201A0020

24451.7077201
Glacier area below 0.1 km2 threshold
Hi

1053 of 1072: 15.10688_CN5O201A0004

248256.136073

Warping all inputs to the following:
Resolution: 10.0
Extent: [684874.3700395847, 3078881.34565086, 685428.4581309519, 3079642.856131122]
Projection: '+proj=utm +zone=45 +datum=WGS84 +units=m +no_defs '
Resampling alg: average

1 of 2: F

In [7]:
df['mult']=df['Area']*df['median']
RGI=df.groupby('RGI_Id')
A=RGI['Area'].agg([np.sum])
A['mult']=RGI['mult'].agg([np.sum])
A['I_dh_dt']=A['mult']/A['sum']
A

,sum,mult,I_dh_dt
RGI_Id,,,
15.02697,478600.0,-3.776547e+05,-0.789082
15.02698,10773600.0,-5.691385e+06,-0.528271
15.02699,2856800.0,1.178592e+06,0.412557
15.027,292500.0,-3.806737e+04,-0.130145
15.02702,1846200.0,2.620261e+05,0.141927
15.02703,56456500.0,-4.145362e+06,-0.073426
15.02704,581600.0,1.008877e+05,0.173466
15.02705,781000.0,1.043512e+04,0.013361
15.02707,1045100.0,-5.877955e+05,-0.562430


#### Calculate the wieghted average dh/dt for every 50 m altitude band taking into account the area of the contributing bands seperately for every glacier
#### Calculate the weighted average dh/dt for the entire watershed taking into account the area of the contributing glaciers

In [9]:
A['m w.e._i']=A['I_dh_dt']*0.85
A['MX']=A['I_dh_dt']*A['sum']
A['MY']=A['m w.e._i']*A['sum']
Regional_dh_dt=(A['MX'].sum())/(A['sum'].sum())
Regional_m_w_e_=(A['MY'].sum())/(A['sum'].sum())
A

,sum,mult,I_dh_dt,m w.e._i,MX,MY
RGI_Id,,,,,,
15.02697,478600.0,-3.776547e+05,-0.789082,-0.670720,-3.776547e+05,-3.210065e+05
15.02698,10773600.0,-5.691385e+06,-0.528271,-0.449031,-5.691385e+06,-4.837678e+06
15.02699,2856800.0,1.178592e+06,0.412557,0.350673,1.178592e+06,1.001803e+06
15.027,292500.0,-3.806737e+04,-0.130145,-0.110623,-3.806737e+04,-3.235727e+04
15.02702,1846200.0,2.620261e+05,0.141927,0.120638,2.620261e+05,2.227222e+05
15.02703,56456500.0,-4.145362e+06,-0.073426,-0.062412,-4.145362e+06,-3.523558e+06
15.02704,581600.0,1.008877e+05,0.173466,0.147446,1.008877e+05,8.575458e+04
15.02705,781000.0,1.043512e+04,0.013361,0.011357,1.043512e+04,8.869854e+03
15.02707,1045100.0,-5.877955e+05,-0.562430,-0.478065,-5.877955e+05,-4.996262e+05


### Mass Budget for the entire watershed

In [10]:
Regional_dh_dt

-0.40307981

In [11]:
Regional_m_w_e_

-0.34261787

### Saving results to csv file

In [13]:
A.to_csv('dh_dt_data_for_glaciers_2008_2015.csv')
#df.to_csv('example.csv')